# RDDの基本

#### [Introduction to Spark with Python, by Jose A. Dianes](https://github.com/jadianes/spark-py-notebooks)

このノートブックでは、3つの基本的なスパーク操作について説明します。そのうち2つは変換マップとフィルタです。もう1つはアクション収集です。同時に、Sparkの永続性の概念を紹介します。

## データの取得とRDDの作成

最初のノートブックで行ったように、KDD Cup 1999に提供された縮小データセット（10％）を使用し、約50万のネットワークインタラクションが含まれます。このファイルは、ローカルでダウンロードするGzipファイルとして提供されています。

In [6]:
import urllib
f = urllib.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

これでRDDを作成することができます。

In [10]:
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)
print(raw_data.count())

494021


## フィルタ変換

この変換は、特定の条件を満たす要素だけを保持するために、RDDに適用できます。 より具体的には、元のRDDのすべての要素について関数を評価する。 新しい結果のRDDには、関数をTrueに戻す要素だけが含まれます。

たとえば、正常な数を数えたいとします。私たちがデータセットに持つ相互作用。 raw_data RDDを次のようにフィルタリングできます。

In [14]:
normal_raw_data = raw_data.filter(lambda x: 'normal.' in x)


In [17]:
from time import time
t0 = time()
normal_count = normal_raw_data.count()
tt = time() - t0
print "There are {} 'normal' interactions".format(normal_count)
print "Count completed in {} seconds".format(round(tt,3))

There are 97278 'normal' interactions
Count completed in 2.118 seconds


これで、新しいRDDにいくつの要素があるのか​​を数えることができます。

ノートブック1から、10％のデータセットに合計494021があることを忘れないでください。 ここでは、97278に正常が含まれていることがわかります。
タグワード。

RDDの要素を数えるための経過時間を測定したことに注目してください。 Sparkの実際の（分散した）計算は、アクションではなく変換を実行するときに行われることを指摘したいので、これを行っています。 この場合、カウントはRDDで実行するアクションです。 私たちは、RDD上で必要な数の変換を適用することができます。この場合、完了するのに数秒かかる最初のアクションを呼び出すまで計算は行われません。

## マップ変換

Sparkのマップ変換を使用することで、RDDのすべての要素に関数を適用できます。 Pythonのラムダは、特にこのような表現に特化しています。

この場合、私たちはデータファイルをCSV形式のファイルとして読みたいと思っています。 RDDの各要素にラムダ関数を適用すると、次のようになります。

In [18]:
from pprint import pprint
csv_data = raw_data.map(lambda x: x.split(","))
t0 = time()
head_rows = csv_data.take(5)
tt = time() - t0
print "Parse completed in {} seconds".format(round(tt,3))
pprint(head_rows[0])

Parse completed in 0.082 seconds
[u'0',
 u'tcp',
 u'http',
 u'SF',
 u'181',
 u'5450',
 u'0',
 u'0',
 u'0',
 u'0',
 u'0',
 u'1',
 u'0',
 u'0',
 u'0',
 u'0',
 u'0',
 u'0',
 u'0',
 u'0',
 u'0',
 u'0',
 u'8',
 u'8',
 u'0.00',
 u'0.00',
 u'0.00',
 u'0.00',
 u'1.00',
 u'0.00',
 u'0.00',
 u'9',
 u'9',
 u'1.00',
 u'0.00',
 u'0.11',
 u'0.00',
 u'0.00',
 u'0.00',
 u'0.00',
 u'0.00',
 u'normal.']


この場合も、最初のSparkアクション（この場合はtake）を呼び出すと、すべてのアクションが発生します。最初の数人だけでなく、たくさんの要素を取るとどうなりますか？

In [19]:
t0 = time()
head_rows = csv_data.take(100000)
tt = time() - t0
print "Parse completed in {} seconds".format(round(tt,3))

Parse completed in 5.148 seconds


時間がかかることがわかります。マップ関数は、RDD上の多くの要素に分散して適用されるため、実行時間が長くなります。

### マップと定義済みの関数の使用

もちろん、あらかじめ定義された関数をmapで使うことができます。 RDD内の各要素を、キーがタグ（たとえばノーマル）であり、値がCSV形式のファイルの行を表す要素のリスト全体であるキーと値のペアとして使用するとします。 我々は次のように進めることができる。

In [20]:
def parse_interaction(line):
    elems = line.split(",")
    tag = elems[41]
    return (tag, elems)

key_csv_data = raw_data.map(parse_interaction)
head_rows = key_csv_data.take(5)
pprint(head_rows[0])

(u'normal.',
 [u'0',
  u'tcp',
  u'http',
  u'SF',
  u'181',
  u'5450',
  u'0',
  u'0',
  u'0',
  u'0',
  u'0',
  u'1',
  u'0',
  u'0',
  u'0',
  u'0',
  u'0',
  u'0',
  u'0',
  u'0',
  u'0',
  u'0',
  u'8',
  u'8',
  u'0.00',
  u'0.00',
  u'0.00',
  u'0.00',
  u'1.00',
  u'0.00',
  u'0.00',
  u'9',
  u'9',
  u'1.00',
  u'0.00',
  u'0.11',
  u'0.00',
  u'0.00',
  u'0.00',
  u'0.00',
  u'0.00',
  u'normal.'])


簡単でしたね。

キーと値のペアを扱うノートブックでは、このタイプのRDDを使用してデータ集約（たとえば、キーでカウント）を行います。

## collectアクション

これまでのところ、conutとtakeを使用しています。 私たちが学ぶ必要があるもう一つの基本的な行動は、mapです。 基本的には、RDD内のすべての要素をメモリに入れて、それらの要素と一緒に作業します。 このため、特に大きなRDDで作業する場合は、注意して使用する必要があります。

生データを使用した例

In [21]:
t0 = time()
all_raw_data = raw_data.collect()
tt = time() - t0
print "Data collected in {} seconds".format(round(tt,3))

Data collected in 11.4 seconds


それは当然以前に使用した他の行動と同じくらい時間がかかりました。 RDDのフラグメントを持つすべてのSparkワーカーノードは、その部分を取り出し、すべてを一緒に減らすために調整する必要があります。

これまでのすべてを組み合わせた最後の例として、すべての正常なやりとりをキーと値のペアとして収集したいと考えています。


In [22]:
# get data from file
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

# parse into key-value pairs
key_csv_data = raw_data.map(parse_interaction)

# filter normal key interactions
normal_key_interactions = key_csv_data.filter(lambda x: x[0] == "normal.")

# collect all
t0 = time()
all_normal = normal_key_interactions.collect()
tt = time() - t0
normal_count = len(all_normal)
print "Data collected in {} seconds".format(round(tt,3))
print "There are {} 'normal' interactions".format(normal_count)

Data collected in 7.688 seconds
There are 97278 'normal' interactions


このカウントは、通常のインタラクションの前のカウントと一致します。 新しい手順はより時間がかかります。 これは、collectですべてのデータを取得し、結果リストにPythonのlenを使用するためです。 以前は、カウントを使用してRDD内の要素の総数を数えていました。